# Coles

In [ ]:
using DataFrames
using Extremes
using Distributions
using Gadfly

### Model validation functions

#### Non-stationary standardization

In [ ]:
function standardize(eva::MaximumLikelihoodEVA)
    dist = Extremes.getdistribution(eva)[1]
    return (1 ./ dist.ξ) .* log.(1 .+ dist.ξ .* (eva.model.data .- dist.μ) ./ dist.σ)
end

function standarddist(::MaximumLikelihoodEVA{BlockMaxima{GeneralizedExtremeValue}})
    return Gumbel()
end
function standarddist(::MaximumLikelihoodEVA{ThresholdExceedance})
    return Exponential()
end


#### Probability plot

In [ ]:
function probabilityplot(eva::MaximumLikelihoodEVA)
    z = sort(eva.model.data)
    m = length(z)
    dist = Extremes.getdistribution(eva)[1]
    
    return probabilityplot(z, m, dist)
end

function residualprobabilityplot(eva::MaximumLikelihoodEVA)
    std = standardize(eva)
    
    z = sort(std)
    m = length(z)
    
    return probabilityplot(z, m, standarddist(eva), title = "Residual Probability Plot")
end
    

function probabilityplot(z::Array{<:Real}, m::Integer, dist::Distribution; title::String = "Probability Plot")
    G̃ = collect(1.0:m) ./ (m + 1)
    Ĝ = cdf.(dist, z)

    l1 = layer(x = G̃, y = Ĝ, Geom.point, Theme(default_color="green"))
    l2 = layer(x = 0.0:0.5:1.0, y = 0.0:0.5:1.0, Geom.line)
    
    return plot(l1, l2, Guide.xlabel("Empirical"), Guide.ylabel("Model"), Guide.title(title))
end

#### Quantile plot

In [ ]:
function quantileplot(eva::MaximumLikelihoodEVA)
    z = sort(eva.model.data)
    m = length(z)
    dist = Extremes.getdistribution(eva)[1]
    
    return quantileplot(z, m, dist)
end

function residualquantileplot(eva::MaximumLikelihoodEVA)
    std = standardize(eva)
    
    z = sort(std)
    m = length(z)
    
    return quantileplot(z, m, standarddist(eva), title = "Residual Quantile Plot")
end

function quantileplot(z::Array{<:Real}, m::Integer, dist::Distribution; title::String = "Quantile Plot")
    Ĝ⁻¹ = quantile.(dist, collect(1.0:m) / (m + 1))

    l1 = layer(x = Ĝ⁻¹, y = z, Geom.point, Theme(default_color="green"))
    l2 = layer(x = Ĝ⁻¹[1]:(Ĝ⁻¹[m] - Ĝ⁻¹[1])/2:Ĝ⁻¹[m], y = Ĝ⁻¹[1]:(Ĝ⁻¹[m] - Ĝ⁻¹[1])/2:Ĝ⁻¹[m], Geom.line)
    
    return plot(l1, l2, Guide.title(title), Guide.xlabel("Model"), Guide.ylabel("Empirical"),
        Coord.cartesian(xmin = Ĝ⁻¹[1], ymin = z[1]))
end

#### Return level plot

In [ ]:
function returnlevelplot(eva::MaximumLikelihoodEVA)
    z = sort(eva.model.data)
    m = length(z)
    dist = Extremes.getdistribution(eva)[1]

    return returnlevelplot(z, m, dist)
end

function returnlevelplot(z::Array{<:Real}, m::Integer, dist::Ty) where Ty<:Distribution
    T = 2:0.2:10
    logT = log.(T)
    p = 1 .- 1 ./ T
    
    q = z[Int.(round.(p .* m))]
    
    layers = []
    push!(layers, layer(x = logT, y = q, Geom.point, Theme(default_color="green")))

    shape = [-0.2, 0.0, 0.2]
    for ξ in shape
        pd = Ty(dist.μ, dist.σ, ξ)
        q = quantile.(pd, p)
        push!(layers, layer(x = logT, y = q, Geom.line))
    end
    
    return plot(layers..., Guide.title("Return Level Plot"), Guide.xlabel("Period"), Guide.ylabel("Level"),
        Coord.cartesian(xmin = logT[1]))
end

#### Density plot

In [ ]:
function densityplot(eva::MaximumLikelihoodEVA)
    z = sort(eva.model.data)
    m = length(z)
    dist = Extremes.getdistribution(eva)[1]

    return densityplot(z, m, dist)
end

function densityplot(z::Array{<:Real}, m::Integer, dist::Distribution)
    nbars = round(sqrt(length(z)))
    start = z[1]
    finish = z[end]
    step = (finish - start) / nbars
    
    function density(v::Real)
        return sum((z .>= (v - step / 2) * ones(m)) .& (z .<= (v + step / 2) * ones(m)))
    end
    
    zb = (start + step/2):step:(finish + step/2)
    db = density.(zb) * 1 / (m * step)
    lb = layer(x = zb, y = db, Geom.BarGeometry)
    
    zl = start:0.01:(finish + step)
    dl = pdf.(dist, zl)
    ll = layer(x = zl, y = dl, Geom.line, Theme(default_color = "Green"))
    
    return plot(ll, lb, Guide.title("Density Plot"), Guide.xlabel("z"), Guide.ylabel("f(z)"))
end

`validationplots` is the function that should be called to display all the graphs 

In [ ]:
function validationplots(eva::MaximumLikelihoodEVA)
    z = sort(eva.model.data)
    m = length(z)
    dist = Extremes.getdistribution(eva)[1]
    
    probabilityPlot = probabilityplot(z, m, dist)
    quantilePlot = quantileplot(z, m, dist)
    returnLevelPlot = returnlevelplot(z, m, dist)
    densityPlot = densityplot(z, m, dist)
    
    return gridstack([probabilityPlot quantilePlot; returnLevelPlot densityPlot])
end

function residualvalidationplots(eva::MaximumLikelihoodEVA)
    std = standardize(eva)

    z = sort(std)
    m = length(z)
    dist = standarddist(eva)
    
    probabilityPlot = probabilityplot(z, m, dist, title = "Residual Probability Plot")
    quantilePlot = quantileplot(z, m, dist, title = "Residual Quantile Plot")
    
    return hstack(probabilityPlot, quantilePlot)
end

### Utility function

`printparams` prints the parameter estimation for a block maxima structure

In [ ]:
function printparams(eva::MaximumLikelihoodEVA)
    for (index, value) in pairs(Extremes.paramindex(eva.model))
        println(index, " : ", eva.θ̂[value])
    end
end

## 3.4.1 Annual Maximum Sea-levels at Port Pirie

In [ ]:
df = load("portpirie")
data = df[:, :SeaLevel];

In [ ]:
gevEVA = gevfit(data)
gev = Extremes.getdistribution(gevEVA)

In [ ]:
validationplots(gevEVA)

10-year return level with 95% accuracy

In [ ]:
year = 10
accuracy = 95 / 100

returnlevel(gevEVA, year, accuracy).cint

## 3.4.2 Glass Fiber Strength Example 

In [ ]:
df = load("glass")
data = -1 * df[:, :Strength]; # minima

In [ ]:
gevEVA = gevfit(data)
gev = Extremes.getdistribution(gevEVA)

In [ ]:
validationplots(gevEVA)

## 4.4.1 Daily Rainfall Data

In [ ]:
df = load("rain")
data = df[:, :Rainfall]

threshold = 30
exceedances = data[data.>threshold] .- threshold

gpEVA = gpfit(exceedances)
gp = Extremes.getdistribution(gpEVA)

In [ ]:
validationplots(gpEVA)

100-year return level with 95% accuracy

In [ ]:
year = 100
mperyear = 365
accuracy = 95 / 100

k = sum(data .> threshold)
n = length(data)

println(returnlevel(gpEVA, threshold, n, mperyear, year, accuracy).cint[1])

## 4.4.2 Dow Jones Index Series

In [ ]:
df = load("dowjones")
data = df[:, :Index]

function dowjonestransformation(i::Integer)
   return 100 * (log(data[i]) - log(data[i - 1]))
end

i = 2:length(data)
transformed = dowjonestransformation.(i)

threshold = 2
exceedances = transformed[transformed.>threshold] .- threshold

gpEVA = gpfit(exceedances)
gp = Extremes.getdistribution(gpEVA)

In [ ]:
validationplots(gpEVA)

## 6.3.1 Annual Maximum Sea-levels

In [ ]:
quantileχ²₁ = quantile(Chisq(1), 0.95)

### Port Pirie

In [ ]:
raw = load("portpirie")
df = DataFrame(raw)
data = df[:, :SeaLevel]

t = ExplanatoryVariable("t", collect(1:length(data)));

#### Stationary

In [ ]:
gevEVA = gevfit(data)
printparams(gevEVA)
gev = Extremes.getdistribution(gevEVA)

mlogls = sum(log.(pdf.(gev, data)));

#### μ linear

In [ ]:
EVA = gevfit(data, locationcov = [t])
printparams(EVA)
linμgev = Extremes.getdistribution(EVA);

mloglμlinear = sum(log.(pdf.(linμgev, data)))

D = 2(mloglμlinear - mlogls)

println()
println("linear μ vs. stationary μ")
println(D, " < ", quantileχ²₁)
println("Stationary is a better represantation")

### Fremantle

In [ ]:
raw = load("fremantle")
df = DataFrame(raw)
data = df[:, :SeaLevel]
year = df[:, :Year]
soi = ExplanatoryVariable("SOI", df[:, :SOI])

t = ExplanatoryVariable("t", collect(1:length(data)))
t2 = ExplanatoryVariable("t2", t.value.^2);

#### Stationary

In [ ]:
gevEVA = gevfit(data)
printparams(gevEVA)
gev = Extremes.getdistribution(gevEVA)

mloglstationary = sum(log.(pdf.(gev, data)));

#### μ linear

In [ ]:
EVA = gevfit(data, locationcov = [t])
printparams(EVA)
linμgev = Extremes.getdistribution(EVA);

mloglμlinear = sum(log.(pdf.(linμgev, data)))

println()
println("linear μ vs. stationary μ")
D = 2(mloglμlinear - mloglstationary)
println(D, " > ", quantileχ²₁)
println("Linear is a better representation")

In [ ]:
t₁ =  1897

l1 = layer(x = year, y = data)

μs = EVA.θ̂[Extremes.paramindex(EVA.model)[:μ]]
println("μ̂  = ", μs)

β̂₀ = μs[1]
β̂₁ = μs[2]
l = β̂₀ .+ β̂₁ .* (year .- t₁)

l2 = layer(x = year, y = l, Geom.line, Theme(default_color = "green"))

plot(l2, l1, Coord.cartesian(xmin = year[1]),
    Guide.title("Fitted estimates for μ"), Guide.xlabel("Year"), Guide.ylabel("Sea-level"))

In [ ]:
residualvalidationplots(EVA)

####  μ quadratic

In [ ]:
EVA = gevfit(data, locationcov = [t, t2])
printparams(EVA)
quadμgev = Extremes.getdistribution(EVA);

mloglμquadratic = sum(log.(pdf.(quadμgev, data)))

println()
println("quadratic μ vs. linear μ")
D = 2(mloglμquadratic - mloglμlinear)
println(D, " < ", quantileχ²₁)
println("Linear is a better representation")

#### σ linear

In [ ]:
EVA = gevfit(data, locationcov = [t], logscalecov = [t])
printparams(EVA)
linσgev = Extremes.getdistribution(EVA);

mloglμlinearσlinear = sum(log.(pdf.(linσgev, data)))

println()
println("linear σ vs. stationary σ")
D = 2(mloglμlinearσlinear - mloglμlinear)
println(D, " < ", quantileχ²₁)
println("Stationary is a better representation")

#### μ SOI

In [ ]:
EVA = gevfit(data, locationcov = [soi])
printparams(EVA)
soiμgev = Extremes.getdistribution(EVA);

mloglSOIμ = sum(log.(pdf.(soiμgev, data)))

println()
println("SOI μ vs. stationary μ")
D = 2(mloglSOIμ - mloglstationary)
println(D, " > ", quantileχ²₁)
println("SOI is a better representation")

#### μ linear + SOI

In [ ]:
EVA = gevfit(data, locationcov = [t, soi])
printparams(EVA)
linsoiμgev = Extremes.getdistribution(EVA);

mloglSOIlinearμ = sum(log.(pdf.(linsoiμgev, data)))

println()
println("linear + SOI μ vs. linear μ")
D = 2(mloglSOIlinearμ - mloglμlinear)
println(D, " > ", quantileχ²₁)
println("Linear + SOI is a better representation")

## 6.3.4 Daily Rainfall Data 

In [ ]:
raw = load("rain")
df = DataFrame(raw)
data = df[:, :Rainfall];

threshold = 30
exceedances = data[data.>threshold] .- threshold

EVA = gpfit(exceedances)
stationarygp = Extremes.getdistribution(EVA)

mloglstationary = sum(log.(pdf.(stationarygp, exceedances)));

In [ ]:
t = ExplanatoryVariable("t", collect(1:length(exceedances)))

EVA = gpfit(exceedances, logscalecov = [t])
printparams(EVA)
linσgp = Extremes.getdistribution(EVA)

mloglσlinear = sum(log.(pdf.(linσgp, exceedances)))

println()
println("linear σ vs. stationary σ")
D = 2(mloglstationary - mloglσlinear)
println(D, " < ", quantileχ²₁)
println("Stationary is a better representation")

## 9.1.3 Example: Port Pirie Annual Maximum Sea-levels 
The priors used by Extremes.jl and by Coles differ, but the results should remain close

In [ ]:
raw = load("portpirie")
df = DataFrame(raw)
data = df[:, :SeaLevel];

EVA = gevfitbayes(data)

In [ ]:
paramin = Extremes.paramindex(EVA.model)
μs = EVA.sim.value[:, paramin[:μ]]
ϕs = EVA.sim.value[:, paramin[:ϕ]]
ξs = EVA.sim.value[:, paramin[:ξ]]
q = quantile(EVA, 0.99)

println("μ̂ = ", mean(μs), " (", sqrt(var(μs)), ")")
println("σ̂ = ", mean(exp.(ϕs)), " (", sqrt(var(exp.(ϕs))), ")")
println("ξ̂ = ", mean(ξs), " (", sqrt(var(ξs)), ")")
println("ẑ = ", mean(q), " (", sqrt(var(q)), ")")